In [9]:
#1.1
import requests
import random
import numpy as np
import sys

url_names = 'https://raw.githubusercontent.com/Micniks/Python-Assignments/master/Week11/names_long.txt'

max_cultist_amount = 6666
min_score = 0
max_score = 50

response = requests.get(url_names)
full_read = ''
for line in response:
    decoded_line = line.decode("utf-8")
    full_read = full_read + decoded_line

all_cultist_names = full_read.split('\n');
cultist_names = []
if(len(all_cultist_names) <= max_cultist_amount): 
    cultist_names = all_cultist_names
else:
    cultist_names = all_cultist_names[:max_cultist_amount]

    
#1.2
def recruit_cultist(name):
    cultist = {name: {
    'stealth': random.randint(min_score, max_score)/10, 
    'influence': random.randint(min_score, max_score)/10, 
    'endurance': random.randint(min_score, max_score)/10, 
    'lore': random.randint(min_score, max_score)/10, 
    'economic': random.randint(min_score, max_score)/10, 
    'strength': random.randint(min_score, max_score)/10, 
    'insanity': random.randint(min_score, max_score)/10}}
    return cultist
    
    
#1.3
def make_cult(cultist_names_list):
    cult = {}
    for cultist in cultist_names_list:
        cult.update(recruit_cultist(cultist))
    return cult

cult = make_cult(cultist_names)


#1.4
cult_roles = {
'Priest': {'stealth': -1, 'influence': 3, 'endurance': 2, 'lore': 4, 'economic': 1, 'strength': 0, 'insanity': 5},
'Enforcer': {'stealth': 1, 'influence': 3, 'endurance': 4, 'lore': 0, 'economic': -1, 'strength': 5, 'insanity': 2},
'Assassin': {'stealth': 5, 'influence': -1, 'endurance': 4, 'lore': 1, 'economic': 0, 'strength': 2, 'insanity': 3},
'Recruiter': {'stealth': 1, 'influence': 5, 'endurance': 0, 'lore': 3, 'economic': 2, 'strength': -1, 'insanity': 4},
'Accountant': {'stealth': 2, 'influence': 1, 'endurance': 3, 'lore': 4, 'economic': 5, 'strength': 0, 'insanity': -1},
'Advisor': {'stealth': 2, 'influence': 4, 'endurance': -1, 'lore': 5, 'economic': 3, 'strength': 0, 'insanity': 1}
}


#Support Method for 1.5
def make_cultist_matrix(data):
    stats = ['stealth', 'influence', 'endurance', 'lore', 'economic', 'strength', 'insanity']
    matrix = []
    names = []
    for name, stats_values in data.items():
        matrix.append(list(stats_values[key] for key in stats))
        names.append(name)
    return {'names': names,'matrix': np.array(matrix)}


#1.5
def assign_cult_roles(cult, cult_roles):
    matrix_cultists_info = make_cultist_matrix(cult)
    cultists_names = matrix_cultists_info['names']
    matrix_C = matrix_cultists_info['matrix']
    matrix_roles_info = make_cultist_matrix(cult_roles)
    roles_names = matrix_roles_info['names']
    matrix_R = matrix_roles_info['matrix'].T
    result_values = matrix_C.dot(matrix_R)
    
    #print('Cultist Values:\n', matrix_C, end='\n\n')
    #print('Roles Modificers:\n', matrix_R, end='\n\n')
    #print('Cultist Role Results:\n', result_values, end='\n\n')
    
    result = {}
    for y_idx in range(0, len(result_values)):
        name = cultists_names[y_idx]
        row = result_values[y_idx]
        highest= {'idx': 0, 'value': -sys.maxsize - 1, 'role': ''}
        for x_idx in range(0,len(row)):
            if(highest['value'] < row[x_idx]):
                highest['idx'] = x_idx
                highest['value'] = row[x_idx]
                highest['role'] = roles_names[x_idx]
        result.update({name: highest['role']})
    return result

def print_role_amount(cultists_roles):
    amounts_for_roles = {}
    print('Total:', len(cultists_roles))
    for role in cult_roles:
        amounts_for_roles.update({role: 0})
    for cultist in cultists_roles:
        role = cultists_roles[cultist]
        amounts_for_roles[role] += 1
    for role in amounts_for_roles:
        print(role + ':', amounts_for_roles[role])

culists_roles = assign_cult_roles(cult, cult_roles)
print_role_amount(culists_roles)
print()

Total: 5063
Priest: 620
Enforcer: 1077
Assassin: 1050
Recruiter: 616
Accountant: 971
Advisor: 729



In [10]:
import csv

culists_roles = assign_cult_roles(cult, cult_roles)

with open('cultists.csv', 'w', newline='') as csvfile:
    c_writer = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    c_writer.writerow(['Cultist', 'Stealth', 'Influence', 'Endurance', 'Lore', 'Economic', 'Strength', 'Insanity', 'Position'])
    for cultist in cult:
        s = cult[cultist]
        c_writer.writerow([cultist, s['stealth'], s['influence'], s['endurance'], s['lore'], s['economic'], s['strength'], s['insanity'], culists_roles[cultist]])

In [11]:
import pandas as pd

data = pd.read_csv('cultists.csv')

data

,Cultist,Stealth,Influence,Endurance,Lore,Economic,Strength,Insanity,Position
0,Nellie Simpson,1.3,2.1,3.3,2.4,3.7,1.2,3.4,Priest
1,Brad Garner,2.3,1.3,2.3,2.6,3.1,4.8,3.0,Enforcer
2,Kristen Watkins,0.7,4.9,1.6,1.0,4.1,5.0,1.6,Enforcer
3,Marcella Peters,1.9,0.0,1.8,1.2,3.8,4.3,0.1,Accountant
4,Vivian Montgomery,1.5,0.5,1.9,2.1,3.6,2.4,1.5,Accountant
...,...,...,...,...,...,...,...,...,...
5058,Martha Harvey,3.6,0.6,3.2,3.8,2.5,3.2,0.8,Accountant
5059,Jovan Watts,1.6,3.0,3.8,4.2,1.5,3.2,3.0,Priest
5060,Jedd Liu,4.4,0.4,0.8,3.4,0.2,1.8,2.6,Assassin
5061,Mia-Rose Wu,1.4,1.2,1.5,2.8,2.6,4.0,4.7,Priest
